In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
plt.style.use('ggplot')

In [ ]:
data = pd.read_csv('User churn.csv')
data.columns = data.columns.str.lower()
data.head()

<b>Encoding binary features</b>

In [ ]:
# Define the mapping dictionary
mapping = {'Yes': 1, 'No': 0}
# Binary features
binary = []
# Apply the mapping to each column where we have only 'Yes' and 'No' values
for col in data.columns:
    if set(data[col].unique()).issubset({'Yes', 'No'}):
        binary.append(col)
        data[col] = data[col].map(mapping)

<b>Visualizing distributions</b>

In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(x = 'churn', y = 'tenure', data = data)
plt.title('Differences in tenure')
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(x = 'churn', y = 'monthlycharges', data = data)
plt.title('Differences in monthly charges')
plt.show()

The probability of churn is higher if monthly charges bigger and less if tenure is longer

<b>Separate categorical and numerical columns</b>

In [ ]:
# Separate the identifier and target variable names as lists
custid = ['customerid']
target = ['churn']
# Separate categorical and numeric column names as lists
categorical = data.nunique()[data.nunique()<10].keys().tolist()
categorical = [x for x in categorical if x not in binary]
binary.remove(target[0])
numerical = [col for col in data.columns if col not in custid + target + categorical + binary]

<b>Standardization</b>

Centers distribution around the mean. Calculates the number of standard deviations away from the mean each point is.

In [ ]:
# Fit the scaler to numerical columns
scaled_numerical = StandardScaler().fit_transform(data[numerical])
# Build a dataframe
scaled_numerical = pd.DataFrame(data=scaled_numerical, columns=numerical)
scaled_numerical.head()

<b>One hot encoding</b>

In [ ]:
# One-hot encoding categorical variables
data = pd.get_dummies(data=data, columns=categorical, drop_first=True)
data.head()

<b>Bringing it all together</b>

In [ ]:
# Drop non-scaled numerical columns
data = data.drop(columns=numerical, axis=1)
# Merge the non-numerical with the scaled numerical data
data = data.merge(right=scaled_numerical,
                     how='left',
                     left_index=True,
                     right_index=True
                    )
data.head()

<b>Dropping correlated features</b>

Highly correlated features can be droped. They provide no additional information to the model.

In [ ]:
data.corr()

In [ ]:
# Compute the correlation matrix
corr_matrix = data.corr().abs()

# Define a threshold for high correlation
threshold = 0.9

# Identify highly correlated features
to_drop = set()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if corr_matrix.iloc[i, j] > threshold:
            colname = corr_matrix.columns[i]
            to_drop.add(colname)
to_drop

In [ ]:
# Drop highly correlated features
data_reduced = data.drop(columns=to_drop)

<b>Churn prediction</b>

Model selection: 1. Logistic regression (simple and interpretable bu can't capture complex relationship) 2. Random forests 3. Support vector machines

<b>Support Vector Classifier</b>

In [ ]:
features = [x for x in data_reduced.columns if x not in custid + target]
X = data_reduced[features]
Y = np.ravel(data_reduced[target])
train_X, test_X,  train_Y, test_Y = train_test_split(X, Y, 
                                                    test_size=0.25,
                                                    random_state=99)
# Print shapes of the datasets
print(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)

In [ ]:
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear', 'poly']  # 'rbf' is used for non-linear, 'linear' for linear, 'poly' for polynomial kernel
}
# Initialize the SVC classifier
svc = SVC(random_state=99)
# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, verbose=1, n_jobs=-1)
# Fit the GridSearchCV
grid_search.fit(train_X, train_Y)
# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")
# Use the best model to make predictions
svc = grid_search.best_estimator_

In [ ]:
svc = grid_search.best_estimator_
pred_Y = svc.predict(test_X)
# Evaluate the model
accuracy = accuracy_score(test_Y, pred_Y)
print(f'Accuracy: {accuracy:.4f}')

In [ ]:
# Print classification report
print(classification_report(test_Y, pred_Y))

In [ ]:
# get confusion matrix
conf_matrix = confusion_matrix(test_Y, pred_Y)
# Plot confusion matrix as a heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Negative', 'Predicted Positive'], yticklabels=['Actual Negative', 'Actual Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


<b>Logistic Regression Classifier</b>

In [ ]:
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']  # 'liblinear' is suitable for small datasets
}
# Initialize the logistic regression model
logreg = LogisticRegression(random_state=99)
# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5, verbose=1, n_jobs=-1)
# Fit the GridSearchCV
grid_search.fit(train_X, train_Y)
# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")
# Use the best model to make predictions
logreg = grid_search.best_estimator_

In [ ]:
# Predict the label
pred_Y = logreg.predict(test_X)
# Evaluate the model
accuracy = accuracy_score(test_Y, pred_Y)
print(f'Accuracy: {accuracy:.4f}')

In [ ]:
# Print classification report
print(classification_report(test_Y, pred_Y))

In [ ]:
# get confusion matrix
conf_matrix = confusion_matrix(test_Y, pred_Y)
# Plot confusion matrix as a heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Negative', 'Predicted Positive'], yticklabels=['Actual Negative', 'Actual Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

<b>Decision Tree Classifier</b>

In [ ]:
# Define the parameter grid
param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2', None]
}
# Initialize the Decision Tree classifier
dtc = DecisionTreeClassifier(random_state=99)
# Initialize GridSearchCV
grid_search_dt = GridSearchCV(estimator=dtc, param_grid=param_grid_dt, cv=5, verbose=1, n_jobs=-1)
# Fit the GridSearchCV
grid_search.fit(train_X, train_Y)
# Get the best parameters
best_params_dt = grid_search_dt.best_params_
print(f"Best parameters found for Decision Tree: {best_params_dt}")
# Use the best model to make predictions
dtc = grid_search_dt.best_estimator_

In [ ]:
# Predict on the test data
pred_Y = dtc.predict(test_X)
# Evaluate the model
accuracy = accuracy_score(test_Y, pred_Y)
print(f'Accuracy: {accuracy:.4f}')

In [ ]:
# Print classification report
print(classification_report(test_Y, pred_Y))

In [ ]:
# get confusion matrix
conf_matrix = confusion_matrix(test_Y, pred_Y)
# Plot confusion matrix as a heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Negative', 'Predicted Positive'], yticklabels=['Actual Negative', 'Actual Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

<b>Random Forest Classifier</b>

In [ ]:
# Define the parameter grid
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}
# Initialize the Random Forest classifier
rfc = RandomForestClassifier(random_state=99)
# Initialize GridSearchCV
grid_search_rf = GridSearchCV(estimator=rfc, param_grid=param_grid_rf, cv=5, verbose=1, n_jobs=-1)
# Fit the GridSearchCV
grid_search_rf.fit(train_X, train_Y)
# Get the best parameters
best_params_rf = grid_search_rf.best_params_
print(f"Best parameters found for Random Forest: {best_params_rf}")
# Use the best model to make predictions
rfc = grid_search_rf.best_estimator_

In [ ]:
# Predict on the test data
pred_Y = rfc.predict(test_X)
# Evaluate the model
accuracy = accuracy_score(test_Y, pred_Y)
print(f'Accuracy: {accuracy:.4f}')

In [ ]:
# Print classification report
print(classification_report(test_Y, pred_Y))

In [ ]:
# get confusion matrix
conf_matrix = confusion_matrix(test_Y, pred_Y)
# Plot confusion matrix as a heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Negative', 'Predicted Positive'], yticklabels=['Actual Negative', 'Actual Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

<b>ROC curve</b>

In [ ]:
# Predict probabilities on the test data
y_pred_prob_sv = svc.predict_proba(test_X)[:, 1]
y_pred_prob_lr = logreg.predict_proba(test_X)[:, 1]
y_pred_prob_dt = dtc.predict_proba(test_X)[:, 1]
y_pred_prob_rf = rfc.predict_proba(test_X)[:, 1]

# Compute ROC curve and AUC for Support Vector
fpr_sv, tpr_sv, _ = roc_curve(test_Y, y_pred_prob_sv)
roc_auc_sv = auc(fpr_sv, tpr_sv)

# Compute ROC curve and AUC for Logistic Regression
fpr_lr, tpr_lr, _ = roc_curve(test_Y, y_pred_prob_lr)
roc_auc_lr = auc(fpr_lr, tpr_lr)

# Compute ROC curve and AUC for Decision Tree
fpr_dt, tpr_dt, _ = roc_curve(test_Y, y_pred_prob_dt)
roc_auc_dt = auc(fpr_dt, tpr_dt)

# Compute ROC curve and AUC for Random Forest
fpr_rf, tpr_rf, _ = roc_curve(test_Y, y_pred_prob_rf)
roc_auc_rf = auc(fpr_rf, tpr_rf)

# Plot ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr_sv, tpr_sv, color='olivedrab', lw=2, label=f'Support Vector (AUC = {roc_auc_sv:.2f})')
plt.plot(fpr_lr, tpr_lr, color='darkred', lw=2, label=f'Logistic Regression (AUC = {roc_auc_lr:.2f})')
plt.plot(fpr_dt, tpr_dt, color='coral', lw=2, label=f'Decision Tree (AUC = {roc_auc_dt:.2f})')
plt.plot(fpr_rf, tpr_rf, color='teal', lw=2, label=f'Random Forest (AUC = {roc_auc_rf:.2f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

Best well-performing model is Logistic Regression in our case.

<b>Principal component analysis</b>

<b>Explained Variance Ratio</b>

A common approach is to choose the number of components that explain a sufficient amount of total variance, typically 90% to 95%.

In [ ]:
# Fit PCA
pca = PCA()
pca.fit(data_reduced.iloc[:, 1:])

# Explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_explained_variance = explained_variance_ratio.cumsum()

# Plot cumulative explained variance
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(cumulative_explained_variance) + 1), cumulative_explained_variance, marker='o', linestyle='--', color='cornflowerblue')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance by Principal Components')
plt.grid(True)
plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=12)
plt.show()

# Determine the number of components that explain at least 95% of the variance
optimal_components = next(x[0] for x in enumerate(cumulative_explained_variance) if x[1] >= 0.95) + 1
print(f'Optimal number of components: {optimal_components}')

<b>Scree Plot</b>

In [ ]:
# Plot explained variance
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, marker='o', linestyle='--', color='cornflowerblue')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Scree Plot')
plt.grid(True)
plt.show()

A scree plot shows the explained variance for each principal component. The "elbow" point in the plot indicates the optimal number of components.

In [ ]:
# Fit PCA
pca = PCA(n_components=18)  # Adjust the number of components to your actual needs
pca.fit(data_reduced.iloc[:, 1:])
# Extract loadings (components)
loadings = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(pca.n_components_)], index=data_reduced.iloc[:, 1:].columns)
# Create a heatmap of loadings
plt.figure(figsize=(10, 6))
sns.heatmap(loadings, annot=True, fmt='.1%', linewidths=1, cmap='coolwarm', annot_kws = {'size': 6})
plt.title('Correlation Heatmap of components')
plt.show()

In [ ]:
# Create meaningful names based on loadings
def interpret_pca_loadings(loadings, top_n=2):
    component_names = []
    for col in loadings.columns:
        # Get the top_n features for each component
        top_features = loadings[col].abs().sort_values(ascending=False).head(top_n).index
        component_name = '+'.join(top_features)
        component_names.append(component_name)
    return component_names

# Generate meaningful names
component_names = interpret_pca_loadings(loadings)
component_names

In [ ]:
# Apply the component names to the principal components DataFrame
principal_components = pd.DataFrame(pca.transform(data_reduced.iloc[:, 1:]), columns=component_names)
principal_components.head()